In [15]:
# input data
import json
import pandas as pd

subject, description = "한국인의 3차원 무릎관절 구축 및 형상 측정", "It is necessary to have a model that describes the feature of the knee Joint with a sufficient accuracy. Koreans, however, do not have their own knee joint model to be used in the total knee replacement arthroplasty. They have to use European or American models which do not match Koreans. Three-dimensional visualization techniques are found to be useful in a wide range of medical applications. Three-dimensional imaging studies such as CT(computed tomography) and MRI(magnetic resonance image) provide the primary source of patient-specific data. Three-dimensional knee joint models were constructed by image processing of the CT data of 10 subjects. Using the constructed model, the dimensions of Korean knee joint were measured. And this study proposed a three-dimensional model and data, which can be helpful to develop Korean knee implants and to analyze knee joint movements."

df_article = pd.read_csv('../data/search_results_article.csv', encoding='UTF-8', low_memory=False)
df_data = pd.read_csv('../data/search_results_dataset.csv', encoding='UTF-8', low_memory=False)

cleaned_df_data = (
    df_data[
        ['svc_id', 'dataset_title_etc_main', 'dataset_expl_etc_main','dataset_pub_dt_pc', 'dataset_kywd_etc_main', 'dataset_creator_etc_main', 'dataset_lndgpg', 'query']
    ]
    .rename(
        columns={
            'svc_id': 'ID',
            'dataset_title_etc_main': 'title',
            'dataset_expl_etc_main': 'description',
            'dataset_pub_dt_pc': 'pubyear',
            'dataset_kywd_etc_main': 'keyword',
            'dataset_creator_etc_main': 'author',
            'dataset_lndgpg': 'URL',
        }
    )
)
cleaned_df_data['category'] = 'dataset'

cleaned_df_arti = (
    df_article[
        ['CN', 'Title', 'Abstract', 'Pubyear', 'Keyword', 'Author', 'ContentURL', 'query']
    ]
    .rename(
        columns={
            'CN': 'ID',
            'Title': 'title',
            'Abstract': 'description',
            'Pubyear': 'pubyear',
            'Keyword': 'keyword',
            'Author': 'author',
            'ContentURL': 'URL'
        }
    )
)
cleaned_df_arti['category'] = 'article'

df = pd.concat([cleaned_df_arti, cleaned_df_data], ignore_index=True)

# 임베딩을 통한 점수 생성 방법

In [ ]:
# text-embedding-3-small 사용

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import pandas as pd
from tqdm import tqdm

df['title'] = df['title'].fillna('')
df['description'] = df['description'].fillna('')

MAX_LENGTH = 1000
df['description'] = df['description'].str.slice(0, MAX_LENGTH)

texts = (df['title'] + " " +
        df['description']).tolist()

# 2. 임베딩 객체 생성
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. Document 객체로 변환
docs = [Document(page_content=text, metadata={"ID": row.ID}) 
        for text, row in zip(texts, df.itertuples())]

batch_size = 500
stores = []
for i in tqdm(range(0, len(docs), batch_size)):
    batch = docs[i:i+batch_size]
    store = FAISS.from_documents(batch, embeddings)
    stores.append(store)

# 부분 벡터스토어 통합
vectorstore = stores[0]
for s in stores[1:]:
    vectorstore.merge_from(s)

# 5. 쿼리 임베딩 생성
query = subject + " " + description
query_embedding = embeddings.embed_query(query)

# 6. 유사 문서 검색
results_with_score = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=20)

# 7. 결과 출력
relevance_df = pd.DataFrame([
    {
        "ID": r.metadata.get("ID"),
        "relevance": score,
    }
    for r, score in results_with_score
]).sort_values(by="relevance", ascending=True)

relevance_df = relevance_df.merge(
    df[['ID', 'title', 'description']],
    on='ID',
    how='left'
)

display(relevance_df)

100%|██████████| 1/1 [00:01<00:00,  1.01s/it]


,ID,relevance,title,description
0,DIKO0010027329,0.048426,한국인의 3차원 무릎 관절 모델 구축 및 형상 측정,It is necessary to have a model that describes...
1,NART73608690,0.484609,Determination of representative dimension para...,<P> Knee joint implants developed by western c...
2,NART23307076,0.654637,"Imaging knee position using MRI, RSA/CT and 3D...",<P><B>Abstract</B></P><P>The purpose of this s...
3,NPAP13526087,0.666785,A 3D Printed Mechanical Model of the Knee to D...,"<P>In this article, a novel 3D printed knee mo..."
4,NART37692760,0.670475,A 3D Model Analysis of Knee Prosthetic Motion ...,<P>The object of this study is to introduce co...
5,f1bd34e52434a57a01b9ab55a6670891,0.712663,무릎 인공관절 형상 설계에 활용할 수 있는 한국인 남녀 무릎 관절의 형상 및 치수 ...,1. 관절면 영상 생성 한국인의 무릎 인공관절 형상 설계에 활용할 수 있는 남녀 ...
6,NART37654746,0.726320,A 3D model analysis for developing multifuncti...,
7,NART74557425,0.728552,Principal component analysis in construction o...,<P>The statistical shape model (SSM) method th...
8,NART40653680,0.745407,Ambulatory measurement of 3D knee joint angle,Three-dimensional measurement of joint motion ...
9,9f0a020d80be42232daacad7a76e6aff,0.764016,Three dimensional point cloud data of Total Kn...,Total Knee Replacement (TKR) is permanent and ...


## 분야별로 나누기

In [16]:
# 제목

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import pandas as pd
from tqdm import tqdm

df['title'] = df['title'].fillna('')

texts = (df['title']).tolist()

# 2. 임베딩 객체 생성
embeddings_title = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. Document 객체로 변환
docs_title = [Document(page_content=text, metadata={"ID": row.ID}) 
        for text, row in zip(texts, df.itertuples())]

batch_size = 500
stores = []
for i in tqdm(range(0, len(docs_title), batch_size)):
    batch = docs_title[i:i+batch_size]
    store = FAISS.from_documents(batch, embeddings_title)
    stores.append(store)

# 부분 벡터스토어 통합
vectorstore = stores[0]
for s in stores[1:]:
    vectorstore.merge_from(s)

# 5. 쿼리 임베딩 생성
query = subject
query_embedding = embeddings_title.embed_query(query)

# 6. 유사 문서 검색
results_with_score = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=len(texts))

# 7. 결과 출력
relevance_df_title = pd.DataFrame([
    {
        "ID": r.metadata.get("ID"),
        "relevance": score,
    }
    for r, score in results_with_score
]).sort_values(by="relevance", ascending=True)

relevance_df_title = relevance_df_title.merge(
    df[['ID', 'category',  'title', 'description']],
    on='ID',
    how='left'
)

display(relevance_df_title)

100%|██████████| 1/1 [00:00<00:00,  1.01it/s]


,ID,relevance,category,title,description
0,DIKO0010027329,0.121606,article,한국인의 3차원 무릎 관절 모델 구축 및 형상 측정,It is necessary to have a model that describes...
1,NPAP13253595,0.591183,article,한국인 사체 무릎관절 형상모델을 이용한 초소형 무릎관절경의 사용 편의성 검증,NaN
2,f1bd34e52434a57a01b9ab55a6670891,0.670177,dataset,무릎 인공관절 형상 설계에 활용할 수 있는 한국인 남녀 무릎 관절의 형상 및 치수 ...,1. 관절면 영상 생성 한국인의 무릎 인공관절 형상 설계에 활용할 수 있는 남녀 ...
3,90fdbc2e1fd69bafb7e5cff6ea228f15,0.819132,dataset,한국인 남성 시신(hm-135)으로부터 얻은 무릎 관절을 움직이는데 필요한 근육 및...,한국인 표준 체형을 가진 남자의 기증 시신 1구를 선정 기준에 맞추어 선정하고 무릎...
4,NART73608690,0.953260,article,Determination of representative dimension para...,<P> Knee joint implants developed by western c...
...,...,...,...,...,...
146,030a9324bc1e54874c049fed1bcdb90d,1.602352,dataset,여론 분석 AI 모델 (2022년 경진대회 우수상),데이터 개요 : * 다양한 매체 속 대중의 의견을 반영하는 여론 분석 시스템의 부재...
147,d23197812f97382655510fd0afb39919,1.609480,dataset,A Zn(II)-based coordination polymer: reduced t...,Based on the mixed ligands <i>p</i>-terphenyl-...
148,7e638f920a29aa83c2b90bd4cbdf401f,1.650010,dataset,MediBioDeBERTa,"데이터 개요 : ![image.png](data:image/png;base64,iV..."
149,9f61784d6ea51e5e3c45e33ecaa3bfe3,1.658542,dataset,Sensory-motor training versus resistance train...,ABSTRACT BACKGROUND: Osteoarthritis of the kne...


In [17]:
# 설명

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import pandas as pd
from tqdm import tqdm

df['description'] = df['description'].fillna('')

MAX_LENGTH = 1000
df['description'] = df['description'].str.slice(0, MAX_LENGTH)

texts = (df['description']).tolist()

# 2. 임베딩 객체 생성
embeddings_description = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. Document 객체로 변환
docs_description = [Document(page_content=text, metadata={"ID": row.ID}) 
        for text, row in zip(texts, df.itertuples())]

batch_size = 500
stores = []
for i in tqdm(range(0, len(docs_description), batch_size)):
    batch = docs_description[i:i+batch_size]
    store = FAISS.from_documents(batch, embeddings_description)
    stores.append(store)

# 부분 벡터스토어 통합
vectorstore = stores[0]
for s in stores[1:]:
    vectorstore.merge_from(s)

# 5. 쿼리 임베딩 생성
query = description
query_embedding = embeddings_description.embed_query(query)

# 6. 유사 문서 검색
results_with_score = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=len(texts))

# 7. 결과 출력
relevance_df_description = pd.DataFrame([
    {
        "ID": r.metadata.get("ID"),
        "relevance": score,
    }
    for r, score in results_with_score
]).sort_values(by="relevance", ascending=True)

relevance_df_description = relevance_df_description.merge(
    df[['ID', 'category',  'title', 'description']],
    on='ID',
    how='left'
)

display(relevance_df_description)

100%|██████████| 1/1 [00:01<00:00,  1.59s/it]


,ID,relevance,category,title,description
0,DIKO0010027329,0.064796,article,한국인의 3차원 무릎 관절 모델 구축 및 형상 측정,It is necessary to have a model that describes...
1,NART73608690,0.447699,article,Determination of representative dimension para...,<P> Knee joint implants developed by western c...
2,NART51664194,0.532292,article,A method of vivo human knee joint 3D model rec...,Objective: To reconstruct a 3D vivo human knee...
3,NART116074894,0.553809,article,Construction of 3D model of knee joint motion ...,<P><B>Abstract</B></P><P>There is a growing de...
4,NART92905321,0.582234,article,Study on 3D Model Reconstruction of Human Knee...,<P>Reconstruction of a 3D model for human knee...
...,...,...,...,...,...
146,NART52043301,1.832493,article,A novel total knee arthroplasty infection mode...,
147,NPAP02242888,1.832493,article,A Numerical Model for Assessment of Total Knee...,
148,NART116936240,1.832493,article,An anatomy-based dynamic model of total knee a...,
149,NART69998616,1.832493,article,A Model for Patellar Resection in Total Knee A...,


In [18]:
# 키워드

from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document
import pandas as pd
from tqdm import tqdm

df['keyword'] = df['keyword'].fillna('')

texts = (df['keyword']).tolist()

# 2. 임베딩 객체 생성
embeddings_keyword = OpenAIEmbeddings(model="text-embedding-3-small")

# 3. Document 객체로 변환
docs_keyword = [Document(page_content=text, metadata={"ID": row.ID}) 
        for text, row in zip(texts, df.itertuples())]

batch_size = 500
stores = []
for i in tqdm(range(0, len(docs_keyword), batch_size)):
    batch = docs_keyword[i:i+batch_size]
    store = FAISS.from_documents(batch, embeddings_keyword)
    stores.append(store)

# 부분 벡터스토어 통합
vectorstore = stores[0]
for s in stores[1:]:
    vectorstore.merge_from(s)

# 5. 쿼리 임베딩 생성
query = description
query_embedding = embeddings_keyword.embed_query(query)

# 6. 유사 문서 검색
results_with_score = vectorstore.similarity_search_with_score_by_vector(query_embedding, k=len(texts))

# 7. 결과 출력
relevance_df_keyword = pd.DataFrame([
    {
        "ID": r.metadata.get("ID"),
        "relevance": score,
    }
    for r, score in results_with_score
]).sort_values(by="relevance", ascending=True)

relevance_df_keyword = relevance_df_keyword.merge(
    df[['ID', 'category',  'title', 'description']],
    on='ID',
    how='left'
)

display(relevance_df_keyword)

100%|██████████| 1/1 [00:01<00:00,  1.15s/it]


,ID,relevance,category,title,description
0,NART74557425,0.604531,article,Principal component analysis in construction o...,<P>The statistical shape model (SSM) method th...
1,NART72011138,0.698641,article,Patient-Specific Computer Model of Dynamic Squ...,Knee forces are highly relevant to performance...
2,NART37692760,0.723299,article,A 3D Model Analysis of Knee Prosthetic Motion ...,<P>The object of this study is to introduce co...
3,NART68187596,0.736034,article,3D reconstruction of bony elements of the knee...,Two separate themes are presented in this pape...
4,NART99506110,0.792440,article,The application of 3D printing patient specifi...,<P><B>Abstract</B></P> <P>The application of ...
...,...,...,...,...,...
146,2ec43729eabbac19f7e764652cf7db4f,1.832493,dataset,"A 3D printed knee brace to improve symptoms, b...",Around 250 millions of people in the world (3....
147,7168051d3d62d30f8c4f2a8e128e47d1,1.832493,dataset,<b>Comparing the effects of transcutaneous ele...,[Purpose] To compare the effects of transcutan...
148,2e3acc355bf4a22c4064c1e61f651ebe,1.832493,dataset,Additional file 1 of Utilization of the Caprin...,Supplementary Material 1
149,d23197812f97382655510fd0afb39919,1.832493,dataset,A Zn(II)-based coordination polymer: reduced t...,Based on the mixed ligands <i>p</i>-terphenyl-...


In [53]:
# 1. 세 개의 데이터프레임 한 번에 병합
merged_df = relevance_df_title.merge(
    relevance_df_description[["ID", "relevance"]], 
    on="ID",
    suffixes=("_title", "_desc")
).merge(
    relevance_df_keyword[["ID", "relevance"]].rename(columns={"relevance": "relevance_key"}), 
    on="ID",
)

# 2. 가중치 합산
a, b, c = 10, 3, 1
merged_df["relevance_raw"] = (
    merged_df["relevance_title"] * a + 
    merged_df["relevance_desc"] * b + 
    merged_df["relevance_key"] * c
) / (a + b + c)

merged_df["relevance_score"] = 100 * (1 - merged_df["relevance_raw"] / 2)

# 3. 필요한 열만 선택 및 정렬
result_df = (merged_df[["ID", "relevance_score", "category", "title", "description"]]
             .rename(columns={"relevance_score": "relevance"})
             .sort_values("relevance", ascending=False)
             .reset_index(drop=True))


display(result_df)

,ID,relevance,category,title,description
0,DIKO0010027329,91.482193,article,한국인의 3차원 무릎 관절 모델 구축 및 형상 측정,It is necessary to have a model that describes...
1,f1bd34e52434a57a01b9ab55a6670891,60.653614,dataset,무릎 인공관절 형상 설계에 활용할 수 있는 한국인 남녀 무릎 관절의 형상 및 치수 ...,1. 관절면 영상 생성 한국인의 무릎 인공관절 형상 설계에 활용할 수 있는 남녀 ...
2,NART73608690,54.613590,article,Determination of representative dimension para...,<P> Knee joint implants developed by western c...
3,NART51664194,53.745998,article,A method of vivo human knee joint 3D model rec...,Objective: To reconstruct a 3D vivo human knee...
4,90fdbc2e1fd69bafb7e5cff6ea228f15,52.731548,dataset,한국인 남성 시신(hm-135)으로부터 얻은 무릎 관절을 움직이는데 필요한 근육 및...,한국인 표준 체형을 가진 남자의 기증 시신 1구를 선정 기준에 맞추어 선정하고 무릎...
...,...,...,...,...,...
146,030a9324bc1e54874c049fed1bcdb90d,21.701496,dataset,여론 분석 AI 모델 (2022년 경진대회 우수상),데이터 개요 : * 다양한 매체 속 대중의 의견을 반영하는 여론 분석 시스템의 부재...
147,5491fb9b38c304c339bd5b3f3e9e83b3,20.340305,dataset,Plasticity of the gastrocnemius elastic system...,Experimental Protocol Animals. To study these ...
148,7e638f920a29aa83c2b90bd4cbdf401f,18.839628,dataset,MediBioDeBERTa,"데이터 개요 : ![image.png](data:image/png;base64,iV..."
149,d23197812f97382655510fd0afb39919,17.989820,dataset,A Zn(II)-based coordination polymer: reduced t...,Based on the mixed ligands <i>p</i>-terphenyl-...


In [ ]:
# 로컬 모델 사용
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain.schema import Document
import pandas as pd

# 검색할 주제와 설명
subject = '기후 데이터 기반의 도시 열섬 현상 분석'
description = '위성 원격탐사 데이터와 국내 기상 관측소 데이터를 결합하여 도시 열섬 현상의 시공간적 패턴을 분석하고, 인공지능 기반 예측 모델을 통해 기후 변화 대응 전략을 모색하는 연구'

# 데이터 로드
processed_df = pd.read_csv('./generated_input_data.csv', encoding='UTF-8', low_memory=False)
vector_df = processed_df[['ID', '제목', '설명', '키워드']]

# 텍스트 결합
texts = (vector_df['제목'] + " " + vector_df['설명'] + " " + vector_df['키워드']).tolist()

# 임베딩 객체 생성
embeddings = HuggingFaceEmbeddings(
    model_name="../model/all-mpnet-base-v2",
    model_kwargs={"device": "cpu"}
)

# Document 객체 생성
docs = [Document(page_content=text, metadata={"id": row.ID}) 
        for text, row in zip(texts, vector_df.itertuples())]

# Chroma 벡터스토어 생성
vectorstore = Chroma.from_documents(
    documents=docs,
    embedding=embeddings,
    # persist_directory="./chroma_db"
)

# 쿼리 생성
query = subject + " " + description

# 유사 문서 20개 검색
k = 20
similar_docs = vectorstore.similarity_search(query, k=k)

# 결과 DataFrame 생성
df_results = pd.DataFrame([
    {
        "ID": doc.metadata.get("id"),
        "제목": vector_df.loc[vector_df['ID'] == doc.metadata.get("id"), "제목"].values[0],
        "설명": vector_df.loc[vector_df['ID'] == doc.metadata.get("id"), "설명"].values[0]
    }
    for doc in similar_docs
])

display(df_results)


,ID,제목,설명
0,46,자율주행을 위한 효율적 알고리즘 연구,자율주행 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험으...
1,57,로보틱스을 위한 효율적 알고리즘 연구,로보틱스 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험으...
2,36,그래프분석을 위한 효율적 알고리즘 연구,그래프분석 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험...
3,37,음성처리을 위한 효율적 알고리즘 연구,음성처리 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험으...
4,52,음성처리을 위한 효율적 알고리즘 연구,음성처리 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험으...
5,54,음성처리을 위한 효율적 알고리즘 연구,음성처리 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험으...
6,55,추천시스템을 위한 효율적 알고리즘 연구,추천시스템 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험...
7,48,컴퓨터비전을 위한 효율적 알고리즘 연구,컴퓨터비전 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험...
8,3,재료과학을 위한 효율적 알고리즘 연구,재료과학 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험으...
9,50,자연어처리을 위한 효율적 알고리즘 연구,자연어처리 분야에서 성능과 계산비용을 동시에 개선한 새로운 알고리즘을 제안하고 실험...
